# Finite Amplitude Impulse Response (FaIR) simple energy balance model demo

This is a simple tutorial demonstrating how to run and modify the FaIR simple energy balance model using different CMIP6 model parameters and/or different forcings and scenarios. 

See the documentation for full details: https://docs.fairmodel.net/en/latest/index.html.

You need to have several python packages installed for this to run; follow the instructions provided in the GitHub repository README.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import tqdm.notebook as tqdm

import fair
from fair import multi_ebm
from fair.energy_balance_model import EnergyBalanceModel


## Part 1: Setup - effective radiative forcings (ERFs)
The key to FaIR is that it translates input forcings into a temperature response. This includes radiative forcings from CO$_2$, CH$_4$, aerosols, volcanoes and more. For this demo we can pull in the forcings from various SSPs. 

Let's start with the AR6-assessed ERFs from SSP1-2.6...

In [ ]:
# .loc[1850:2100] lets us focus on preindustrial through 21st century warming, but the forcing files go from 1750 to 2500.

df_forcing_126 = pd.read_csv('files/ERF_ssp126_1750-2500.csv', index_col='year').loc[1850:2100]
df_forcing_126


You will often see total aerosol radiative forcing grouped together (ERFaer), which is the sum of ERF from aerosol-radiation (ERFari) and ERF from aerosol-cloud interactions (ERFaci). Let's add a column to the forcings data frame for the total effective radiative forcing from aerosol:

In [ ]:
# ERFaer = ERFaci + ERFari
df_forcing_126['aerosol'] = df_forcing_126['aerosol-cloud_interactions'] + df_forcing_126['aerosol-radiation_interactions']
df_forcing_126


We can plot individual components of the forcings to see how they vary over time. Do we expect aerosol forcing to provide a cooling or warming effect?

In [ ]:
# Quick plot - looking all total (ALL) forcing, aerosol forcing, anthropogenic forcing, and methane (ch4)

df_forcing_126[['total', 'aerosol', 'total_anthropogenic', 'ch4']].plot()
plt.gca().axhline(0, color='k', lw=0.5)
plt.title("SSP1-2.6")
plt.show()


SSP 1-2.6 is one of the lowest/most conservative emissions scenarios. We can also try a more moderate one, like SSP2-4.5.

In [ ]:
df_forcing_245 = pd.read_csv('files/ERF_ssp245_1750-2500.csv', index_col='year').loc[1850:2100]
df_forcing_245


In [ ]:
# Same as before - make a category for total aerosol radiative forcing: ERFaer = ERFaci + ERFari
df_forcing_245['aerosol'] = df_forcing_245['aerosol-cloud_interactions'] + df_forcing_245['aerosol-radiation_interactions']
df_forcing_245


How do the forcings from SSP 1-2.6 and SSP2-4.5 compare?

In [ ]:
fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(12, 4))

# SSP 1-2.6
ax1.plot(df_forcing_126['total'], color="C0", label="total")
ax1.plot(df_forcing_126['aerosol'], color="C1", label="aerosol")
ax1.plot(df_forcing_126['total_anthropogenic'], color="C2", label="total_anthropogenic")
ax1.plot(df_forcing_126['ch4'], color="C3", label="ch4")
ax1.axhline(0, color='k', lw=0.5)
ax1.set_title("SSP1-2.6")

# SSP 2-4.5
ax2.plot(df_forcing_245['total'], color="C0", label="total")
ax2.plot(df_forcing_245['aerosol'], color="C1", label="aerosol")
ax2.plot(df_forcing_245['total_anthropogenic'], color="C2", label="total_anthropogenic")
ax2.plot(df_forcing_245['ch4'], color="C3", label="ch4")
ax2.axhline(0, color='k', lw=0.5)
ax2.set_title("SSP2-4.5")

for ax in [ax1, ax2]:
    ax.legend(loc="upper left")
    ax.set_ylim(-2, 6) # same y-axis limits for a fair comparison (no pun intended)
    
plt.show()


There are other SSPs we can try later on:

In [ ]:
# list the names of the SSP ERF files we have in this repository
! ls files/*ssp*

## Part 2: Setup - coupled model parameters
Under the hood, the FaIR model uses a lot of input parameters taken from the fully-coupled CMIP6 models. These parameters tell FaIR, for example, how sensitivity it should be to a quadrupling of CO$_2$ (`F_4xCO2`), what the heat capacity of the ocean layers are (`C1`, `C2`, `C3`), and so on. The input file has parameters for dozens of different CMIP6 models, some with multiple runs. 

In [ ]:
# physical model parameters
all_model_params = pd.read_csv('files/4xCO2_cummins_ebm3.csv',index_col=['model'])
all_model_params


In [ ]:
# Let's pick one model to start with: CESM2
cesm2_params = all_model_params.loc['CESM2']
cesm2_params


## Part 3: Running the model
We want to use FaIR's `EnergyBalanceModel()` class to convert the input forcings into a past and future temperature. 

In [ ]:
# How do we set this up? Let's check the documentation:
EnergyBalanceModel?


First, we need to initialize the model:

In [ ]:
# We can use the CESM2 parameters for each of the FaIR input parameters

ebm3 = EnergyBalanceModel(
    ocean_heat_capacity=[cesm2_params.C1, cesm2_params.C2, cesm2_params.C3],
    ocean_heat_transfer=[cesm2_params.kappa1, cesm2_params.kappa2, cesm2_params.kappa2],
    deep_ocean_efficacy=cesm2_params.epsilon,  
    gamma_autocorrelation=cesm2_params.gamma,  
    sigma_xi=cesm2_params.sigma_xi,
    sigma_eta=cesm2_params.sigma_eta,
    forcing_4co2=cesm2_params.F_4xCO2,

    # Set this to True to add in a stochastic internal variability component
    stochastic_run=False,

    # We want to set a seed for reproduciblity. This can be any number.
    seed=16
)


Now we need to give the FaIR model some input forcings:

In [ ]:
# An easy place to start is to use the total forcing from SSP1-2.6 - everything that goes into AR6
ebm3.add_forcing(forcing = df_forcing_126['total'].values, timestep=1)


We still haven't run it though - it gives us nothing if we try to look at the output temperature.

In [ ]:
ebm3

In [ ]:
ebm3.temperature

All we need to do is call `.run()` - then we get something exciting for temperature:

In [ ]:
ebm3.run()

# access the temperature output
print(ebm3.temperature)

# the temperature output is in (year, layer)
print(np.shape(ebm3.temperature))


Now that we have temperature output, we can plot it!

The layer at index zero is the surface/top ocean layer, which dominates GMST - that's the layer we care about here.


In [ ]:
# First put the output temperature projections into a data frame for easier plotting/analysis
ebm_tas = pd.Series(data=ebm3.temperature[:,0], index=df_forcing_126.index)


In [ ]:
ebm_tas.plot()

## Part 4: Experiments


#### Define some functions for convenience

To make messing with the output easier, let's make a function where we can input the total forcing and model parameters and output the temperature time series. (You can also just copy/paste the body of this function into a new cell and directly edit/run).

In [ ]:
def run_fair(forcing=df_forcing_126['total'], model_params=all_model_params.loc['CESM2']):
    """ 
    Run FaIR to get a projected temperature change for some model parameters
    and some input forcing. Defaults to CESM2 and the SSP1-2.6 total forcing.
    """
    ebm = EnergyBalanceModel(
            ocean_heat_capacity=[model_params.C1, model_params.C2, model_params.C3],
            ocean_heat_transfer=[model_params.kappa1, model_params.kappa2, model_params.kappa2],
            deep_ocean_efficacy=model_params.epsilon, 
            gamma_autocorrelation=model_params.gamma,  
            sigma_xi=model_params.sigma_xi,
            sigma_eta=model_params.sigma_eta,
            forcing_4co2=model_params.F_4xCO2,
            stochastic_run=False,
            seed=16
        )

    ebm.add_forcing(forcing=forcing.values, timestep=1)
    ebm.run()
    
    ebm_tas = pd.Series(data=ebm.temperature[:, 0], index=forcing.index)

    return ebm_tas
    

In [ ]:
# This should give us the same thing as before, with SSP1-2.6 and CESM2 parameters:
ebm_tas_cesm2 = run_fair()
ebm_tas_cesm2.plot()


#### Some things you could try
What happens if you...
* use parameters from other CMIP6 models? How big is the spread in predictions?
* take away methane? If you take away CO2?
* change the SSP?

In [ ]:
# Same forcings, different model
ebm_tas_hadgem = run_fair(model_params=all_model_params.loc["HadGEM3-GC31-MM"])

plt.plot(ebm_tas_hadgem, label="HadGEM3-GC31-MM")
plt.plot(ebm_tas_cesm2, label="CESM2")
plt.legend()
plt.show()


In [ ]:
# What if we take away methane?
# We need to give FaIR input for ALL forcings except methane (CH4)

forcing_noch4 = df_forcing_126['total'] - df_forcing_126['ch4']

ebm_tas_cesm2_noch4 = run_fair(forcing=forcing_noch4)

plt.plot(ebm_tas_cesm2_noch4, label="No methane")
plt.plot(ebm_tas_cesm2, label="All forcings")
plt.legend()
plt.show()
